In [39]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import cv2
from google.colab import files
import os
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, Conv2DTranspose, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape,LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature
from google.colab import drive

In [40]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d tanlikesmath/the-oxfordiiit-pet-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
the-oxfordiiit-pet-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [41]:
!unzip "/content/the-oxfordiiit-pet-dataset.zip" -d "/content/dataset/"

Archive:  /content/the-oxfordiiit-pet-dataset.zip
replace /content/dataset/images/Abyssinian_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [42]:
!tar -xvzf "/content/annotations.tar.gz"

annotations/
annotations/list.txt
annotations/README
annotations/test.txt
annotations/trainval.txt
annotations/._trimaps
annotations/trimaps/
annotations/xmls/
annotations/xmls/Abyssinian_1.xml
annotations/xmls/Abyssinian_10.xml
annotations/xmls/Abyssinian_100.xml
annotations/xmls/Abyssinian_101.xml
annotations/xmls/Abyssinian_102.xml
annotations/xmls/Abyssinian_103.xml
annotations/xmls/Abyssinian_105.xml
annotations/xmls/Abyssinian_106.xml
annotations/xmls/Abyssinian_107.xml
annotations/xmls/Abyssinian_108.xml
annotations/xmls/Abyssinian_109.xml
annotations/xmls/Abyssinian_11.xml
annotations/xmls/Abyssinian_110.xml
annotations/xmls/Abyssinian_111.xml
annotations/xmls/Abyssinian_112.xml
annotations/xmls/Abyssinian_113.xml
annotations/xmls/Abyssinian_114.xml
annotations/xmls/Abyssinian_115.xml
annotations/xmls/Abyssinian_116.xml
annotations/xmls/Abyssinian_117.xml
annotations/xmls/Abyssinian_118.xml
annotations/xmls/Abyssinian_119.xml
annotations/xmls/Abyssinian_12.xml
annotations/xmls/

In [54]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__ (self, images, maps, batch_size, INPUT_DIM, shuffle = False):

        self.images = images
        self.maps = maps
        self.batch_size = batch_size
        self.train_image_list=os.listdir(images)
        self.INPUT_DIM=INPUT_DIM

    def __len__(self):
        return int(np.floor(len(self.train_image_list)/self.batch_size))

    def __getitem__(self, idx):
        x,y = self.__data_generation(idx)
        y-=1
        return np.array(x), np.array(y)

    def __data_generation(self, idx):
        x = []
        y = []

        for j in range(idx*self.batch_size, (idx+1)*self.batch_size):
            img_array = img_to_array(load_img(self.images+os.listdir(self.images)[j],target_size=(self.INPUT_DIM,self.INPUT_DIM)))
            map_array = img_to_array(load_img(self.maps+os.listdir(self.maps)[j], color_mode='grayscale', target_size=(self.INPUT_DIM//2,self.INPUT_DIM//2)))

            X.append(img_array)
            y.append(map_array)

            X.append(tf.image.adjust_saturation(img_array, 0.5))
            y.append(map_array)

            X.append(tf.image.flip_up_down(img_array))
            y.append(tf.reverse(map_array, [0]))
        return tf.convert_to_tensor(x),tf.convert_to_tensor(y)

In [44]:
train_images='/content/dataset/images/'
train_maps='/content/dataset/trimaps/'

LR=1e-4
BATCH_SIZE=4
EPOCH=100
INPUT_DIM=224
NUM_FILTERS=128

In [45]:
train_gen = DataGenerator(train_images, train_maps,BATCH_SIZE,INPUT_DIM)

In [46]:
def get_base_model():
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        input_shape=(INPUT_DIM,INPUT_DIM,3),
        include_top=False,)
    base_model.trainable=False

    conv1_relu,conv2_block3_out,conv3_block4_out,conv4_block6_out,conv5_block3_out=[base_model.get_layer(layer_name).output for layer_name in ["conv1_relu","conv2_block3_out","conv3_block4_out","conv4_block6_out","conv5_block3_out"]]

    return Model(
        inputs=[base_model.inputs],outputs=[conv1_relu,conv2_block3_out,conv3_block4_out,conv4_block6_out,conv5_block3_out]
    )

get_base_model().summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_8[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [47]:
class Upsample(tf.keras.layers.Layer):
    def __init__(self,NUM_FILTERS):
        super(Upsample,self).__init__()
        self.conv_t_1=Conv2DTranspose(NUM_FILTERS,1,strides=2,activation='relu')
        self.norm_1=BatchNormalization()
    def call(self,x):
        x=self.norm_1(self.conv_t_1(x))
        return x

In [48]:
class ConvLayers(tf.keras.layers.Layer):
    def __init__(self,NUM_FILTERS):
        super(ConvLayers,self).__init__()
        self.conv_1=Conv2D(NUM_FILTERS*2,3,padding='same',activation='relu')
        self.norm_1=BatchNormalization()

        self.conv_2=Conv2D(NUM_FILTERS*4,3,padding='same',activation='relu')
        self.norm_2=BatchNormalization()
    def call(self,x):
        x=self.norm_1(self.conv_1(x))
        x=self.norm_2(self.conv_2(x))
        return x

In [56]:
inputs=tf.keras.Input(shape=(INPUT_DIM,INPUT_DIM,3))
x=Rescaling(1/255.)(inputs)
x_112,x_56,x_28,x_14,x_7=get_base_model()(x)

x=Upsample(NUM_FILTERS)(x_7)
x=tf.concat([x,x_14],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)
x=Dropout(0.5)(x)

x=Upsample(NUM_FILTERS)(x)
x=tf.concat([x,x_28],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)
x=Dropout(0.5)(x)

x=Upsample(NUM_FILTERS)(x)
x=tf.concat([x,x_56],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)
x=Dropout(0.5)(x)

x=Upsample(NUM_FILTERS)(x)
x=tf.concat([x,x_112],axis=-1)
x=ConvLayers(NUM_FILTERS)(x)

out=Conv2D(3,3,padding='same',activation='softmax')(x)
model=tf.keras.Model(inputs=inputs,outputs=out)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_11[0][0]']            
                                                                                                  
 model_8 (Functional)        [(None, 112, 112, 64),       2358771   ['rescaling_4[0][0]']         
                              (None, 56, 56, 256),        2                                       
                              (None, 28, 28, 512),                                                
                              (None, 14, 14, 1024),                                         

In [ ]:
def custom loss (y_true, y_pred):

  cce = tf.keras.losses.SparseCategoricalCrossentropy()
  y_true_background = tf.gather_nd(y true, tf.where(y true[..., 0]== 1))
  y_pred_background= tf.gather_nd(y_pred, tf.where(y_true[..., 0]== 1))
  y_true boundary= tf.gather_nd(y_true, tf.where(y true[..., 0] == 2))
  y_pred boundary= tf.gather_nd(y_pred, tf.where(y_true[...] == 2))
  y_true_object= tf.gather_nd(y_true, tf.where(y_true[..., 0] == 0))
  y_pred_object = tf.gather_nd(y_pred, tf.where(y_true[..., 0] == 0))

  cce_background= cce(y_true_background, y_pred_background)
  cce boundary= cce(y_true boundary, y_pred_boundary)
  cce_object= cce(y_true_object, y_pred_object)

  FACTOR_BACKGROUND, FACTOR_BOUNDARY, FACTOR_OBJECT = 1.0, 3.0, 2.0

  cce_total = FACTOR_BACKGROUND*cce_background+ FACTOR_BOUNDARY*cce_boundary FACTOR_OBJECT*cce_object

  return cce total

In [57]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(learning_rate = LR),
    metrics='accuracy',
    run_eagerly = True,
)

In [58]:
checkpoint_filepath='/content/drive/MyDrive/Bang/image_segmentation.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

In [ ]:
history = model.fit(
    train_gen,
    verbose=1,
    shuffle=True,
    epochs=EPOCH,
    callbacks=[callback])

In [ ]:
test_image =''
test_image_map=''

X=[]
X.append(img_to_array(load_img(test_image,target_size=(224,224))))
image_output=tf.argmax(model.predict(tf.constant(X)),axis=-1)[0]
image_output=tf.expand_dims(image_output,axis=-1)

In [ ]:
image=tf.keras.preprocessing.image.load_img(test_image,color_mode='rgb',target_size=(224,224))
plt.imshow(image)
plt.show()

In [ ]:
plt.imshow(image_output[...,0])
plt.show()